In [ ]:
import math

In [3]:
# Identify bits

bits = [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]

56


In [12]:
# Decode ASCII characters

def decode(bits):
    if len(bits)%7 != 0:
        return "Decoding error 1: Not ASCII sequence."
    bit_strings = []
    for i in range(int(len(bits)/7)):
        bit_string = ''.join([str(bits[7*i + n]) for n in range(7)])
        bit_strings.append(bit_string)
    
    letters = []
    for bitstr in bit_strings:
        num = int('0b' + bitstr, 2)
        letters.append(chr(num))

    return ''.join(letters)
decode(bits)

'NCC-1701'